In [ ]:
import os
import time
import numpy as np 
import pandas as pd
from itertools import product
import joblib
import warnings


warnings.simplefilter('ignore')
pd.options.display.max_columns = 50


def preprocess_sales(sales):
    sales['weekofyear'] = sales['date'].dt.weekofyear.astype(np.uint8)
    sales['quater'] = sales['date'].dt.quarter.astype(np.uint8)
    sales['day'] = sales['date'].dt.day.astype(np.uint8)
    sales['sell_price'] = sales['sell_price'].fillna(0)
    sales['event_type_1'] = sales['event_type_1'].cat.add_categories("unknown").fillna("unknown")
    sales['event_type_2'] = sales['event_type_2'].cat.add_categories("unknown").fillna("unknown")
    sales['event_name_1'] = sales['event_name_1'].cat.add_categories("unknown").fillna("unknown")
    sales['event_name_2'] = sales['event_name_2'].cat.add_categories("unknown").fillna("unknown")
    return sales


def read_data(is_train=True, first_day=1, last_day=1913, observation_window=57, 
             calendar_file=None, prices_file=None, sales_file=None):
    """
    is_train: признак, мы считываем данные для обучения модели или submission
    first_day: с какого дня читаем данные (например, первые 300 дней можем пропустить).
    Акутально только, если is_train = True
    last_day: последний день в выборке 
    observation_window: окно наблюдения, за которое нам нужно прочитать данные (актуально только
    если is_train = False. Например, для имплементации нам нужны данные только за 50 дней назад)
    calendar_file, prices_file, sales_file: путь к файлам с данными
    """
    start_day = first_day if is_train else last_day - observation_window
    finish_day = min(1913, last_day)

    daycols = ['d_{}'.format(i) for i in range(start_day, finish_day + 1)]
    daycols = {c: np.int16 for c in daycols}

    catcols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    catcols = {c: 'category' for c in catcols}
    sales_dtypes = {**catcols, **daycols}

    calendar_dtypes = dict(
          wm_yr_wk=np.int16, 
          weekday='category', 
          wday=np.uint8, 
          month=np.uint8,
          year=np.int16,
          event_name_1='category', 
          event_type_1='category', 
          event_name_2='category', 
          event_type_2='category', 
          snap_CA=np.uint8, 
          snap_TX=np.uint8, 
          snap_WI=np.uint8
    )

    prices_dtypes = dict(
         store_id='category', 
         item_id='category', 
         wm_yr_wk=np.int16, 
         sell_price=np.float32
    )

    calendar = pd.read_csv(calendar_file, dtype=calendar_dtypes)
    calendar['date'] = pd.to_datetime(calendar['date'])
    calendar['d'] = calendar['d'].str[2:10].astype(np.int16)

    sell_prices = pd.read_csv(prices_file, dtype=prices_dtypes)

    sales = pd.read_csv(sales_file, dtype=sales_dtypes)

    # add empy day sales for test data we should predict
    if not is_train:
        for day in range(last_day + 1, last_day + 29):
            daycols[f"d_{day}"] = np.int16
            sales[f"d_{day}"] = np.nan

    sales = pd.melt(sales, id_vars=catcols, value_vars=daycols, var_name="d", value_name="sales")
    sales['d'] = sales['d'].str[2:10].astype(np.int16)
    print("Max day", sales['d'].max())

    print("Sales shape before merge calendar", sales.shape)
    sales = sales.merge(calendar, on='d', how='left', copy=False)
    print("Sales shape after merge calendar", sales.shape)
    sales = sales.merge(sell_prices, on=["store_id", "item_id", "wm_yr_wk"], how='left', copy=False)
    sales['store_id'] = sales['store_id'].astype('category')
    sales['item_id'] = sales['item_id'].astype('category')
    print("Sales shape after merge prices", sales.shape)

    # some date based features and preprocessing
    sales = preprocess_sales(sales)
    
    return sales


def read_train_data(use_cache=True, first_day=1, last_day=1913):
    joblib_filename='/kaggle/input/combine-train-data/sales_train_validation.joblib'
    if os.path.isfile(joblib_filename) and use_cache:
        sales = joblib.load(joblib_filename)
        sales = preprocess_sales(sales)        
        return sales
    return read_data(is_train=True, first_day=first_day, last_day=last_day, 
                     calendar_file='/kaggle/input/m5-forecasting-accuracy/calendar.csv', 
                     prices_file='/kaggle/input/m5-forecasting-accuracy/sell_prices.csv', 
                     sales_file='/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv'
                    )


def read_test_data(last_day=1913, obesrvation_window=57):
    return read_data(is_train=False, last_day=last_day, observation_window=obesrvation_window,
                     calendar_file='/kaggle/input/m5-forecasting-accuracy/calendar.csv', 
                     prices_file='/kaggle/input/m5-forecasting-accuracy/sell_prices.csv', 
                     sales_file='/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv'
                    )


def feature_engineering(df, shifts_size=None, windows_size=None):

    for shift_size in shifts_size:
        
        sales_shift_col = f'sales_shift_{shift_size}D'
        sell_price_shift_col = f'sell_price_shift_{shift_size}D'
    
        shifted_df = df[['id', 'sales', 'sell_price']].groupby('id')[['sales', 'sell_price']].shift(shift_size)
        df[sales_shift_col] = shifted_df['sales'].fillna(0).astype(np.float32)
        df[sell_price_shift_col] = shifted_df['sell_price'].fillna(0).astype(np.float32)
        
        for window_size in windows_size:        

            rolled_features = df[['id', sales_shift_col, sell_price_shift_col]].rolling(window_size).agg({
                sales_shift_col: ['mean', 'std'], 
                sell_price_shift_col: ['min', 'mean']                
            })

            rolled_features.columns = ['_'.join(c) + f'_{window_size}D' for c in rolled_features]
            rolled_features.fillna(0, inplace=True)
            rolled_features = rolled_features.apply(lambda x: x.astype(np.float32))

            df = pd.concat([df, rolled_features], axis=1)
            del rolled_features
        
        del shifted_df
    
    return df


def get_weights_validation(data):
#     https://raw.githubusercontent.com/Mcompetitions/M5-methods/master/validation/weights_validation.csv
    last_28_days = data[data['d'] >= 1886]
    last_28_days['revenue'] = last_28_days['sales'] * last_28_days['sell_price']

    item_weights = []
    last_28_days['total'] = 'Total'
    last_28_days['x'] = 'X'
    levels = {
        'Level1': ['total', 'x'], 
        'Level2': ['state_id', 'x'], 
        'Level3': ['store_id', 'x'], 
        'Level4': ['cat_id', 'x'], 
        'Level5': ['dept_id', 'x'], 
        'Level6': ['state_id', 'cat_id'], 
        'Level7': ['state_id', 'dept_id'], 
        'Level8': ['store_id', 'cat_id'], 
        'Level9': ['store_id', 'dept_id'], 
        'Level10': ['item_id', 'x'], 
        'Level11': ['state_id', 'item_id'], 
        'Level12': ['item_id', 'store_id']
    }
    for level_id, aggrs in levels.items():
        aggr_1, aggr_2 = aggrs

        sales_aggr = last_28_days.groupby([aggr_1, aggr_2]).agg(dict(revenue='sum')) / last_28_days['revenue'].sum()
        sales_aggr.reset_index(inplace=True)
        sales_aggr.columns = ['Agg_Level_1', 'Agg_Level_2', 'Weight']
        sales_aggr['Level_id'] = level_id
        sales_aggr = sales_aggr[['Level_id', 'Agg_Level_1', 'Agg_Level_2', 'Weight']]
        item_weights.append(sales_aggr)

    item_weights = pd.concat(item_weights)
    item_weights.reset_index(drop=True, inplace=True)
    return item_weights


def get_weights(data, min_d, max_d=1913):
    sample = data[data['d'].between(min_d, max_d)]
    sample['revenue'] = sample['sales'] * sample['sell_price']
    weights = sample.groupby('id').agg(dict(revenue='sum')) / sample['revenue'].sum()
    weights.sort_index(inplace=True)
    weights = np.array(weights['revenue'])
    return weights


def get_denominator(data, min_d, max_d=1913):
    data_short = data[['id', 'sales', 'd']]
    data_short = data_short[data_short['d'].between(min_d, max_d)]
    data_short['sales_prev'] = data_short[["id","sales"]].groupby("id")["sales"].shift(1)
    data_short = data_short[data_short['sales'] > 0] 
    data_short['squared_diff'] = (data_short['sales'] - data_short['sales_prev']) ** 2
    denominator = data_short.groupby('id').agg(dict(squared_diff='mean'))
    denominator.sort_index(inplace=True)
    denominator = np.array(denominator['squared_diff'])
    return denominator    


def get_numerator(y_true, y_pred, cnt_items=30490, cnt_days=28):
    assert len(y_true) == len(y_pred)
    
    cnt_rows = int(cnt_items * cnt_days)
    y_true = y_true[-cnt_rows:]
    y_pred = y_pred[-cnt_rows:]

    diff = (y_true - y_pred) ** 2
    diff = diff.reshape(cnt_items, cnt_days)
    return diff.mean(axis=1)


class WRMSSE:
    def __init__(self, weights, denominator, cnt_items=30490, cnt_days=28):
        self.weights = weights
        self.denominator = denominator
        self.cnt_items = cnt_items
        self.cnt_days = cnt_days
                
    def __call__(self, y_true, y_pred):
        numerator = get_numerator(y_true, y_pred, cnt_items=self.cnt_items, cnt_days=self.cnt_days)
        rmsse = ((numerator / self.denominator) ** 0.5)
        return np.dot(self.weights.reshape(1, -1), rmsse.reshape(-1, 1))[0][0]

Time Series Deep Learning

In [ ]:
%%time


df = read_train_data(use_cache=True)
df.head()

In [ ]:
df.shape

In [ ]:
df = df.loc[df['store_id'] == 'CA_1']

In [ ]:
df.shape

In [ ]:
import random
import pandas as pd
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tensorflow import keras 
from tensorflow.keras import backend as K
from sklearn.metrics import mean_absolute_error, mean_squared_error

%matplotlib inline

In [ ]:
# make sure you run on tensorlfow 2.2.0. This notebook won't work on 2.1.0 ;(
tf.__version__

In [ ]:
# seed random generators
# don't suggest to do it in production 
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Helper functions 

#### First, let's prepare a code which generates sliding windows. 
We are going to use numpy index_tricks for that. Function  takes original data and 

In [ ]:
# создаем 3 мерные x,y
def create_sliding_windows(arr, window_size):
    """ Takes original data matrix with shape (N, D) and creates sliding windows of size window_size. 
    Result matrix has shape (M, W, D), where
        - M is number of observations equal to (N - window_size + 1)
        - W is window_size
        - D - dimensinality of the original feature space
        
    Parameters
    ----------
    arr : np.ndarray 
        Original 2D matrix
    window_size : int
        Size of the sliding window
    
    Returns
    -------
    result : np.ndarray 
        Resulting 3D matrix where each observation is a sliding window. 
    """
    arr = arr.astype(np.float)
    (stride,) = arr.strides
    return np.lib.index_tricks.as_strided(
        arr,
        (arr.shape[0] - window_size + 1, window_size),
        strides=[stride, stride],
        writeable=False,
    )


# other helper functions for plotting losses and predictions later
def plot_loss(history): 
    plt.figure(figsize=(10, 5))
    loss = history['loss']
    line = plt.plot(range(len(loss)), loss)
    line[0].set_label('Training loss')
    if 'val_loss' in history: 
        val_loss = history['val_loss']
        line = plt.plot(range(len(val_loss)), val_loss)
        line[0].set_label('Validation loss')
    plt.legend()
    
    #как харашо обучаем
def plot_predictions(x, y_true, y_pred):
    plt.figure(figsize=(10, 5))
    x_indexes = np.arange(x.shape[0])
    y_indexes = np.arange(x.shape[0], x.shape[0] + y_true.shape[0])
    line1 = plt.plot(x_indexes, x)
    line2 = plt.plot(y_indexes, y_true)
    line3 = plt.plot(y_indexes, y_pred)
    
    line1[0].set_label('Historical data')
    line2[0].set_label('Ground truth')
    line3[0].set_label('Forecast')
    plt.legend()
    
    
class MeanScaler(object):
    def __init__(self, copy=True):
        self.copy = copy
        self.scale_ = None
        
    def fit(self, X): 
        self.scale_ = np.mean(np.abs(X), axis=1, keepdims=True) + 1
        return self
    
    def _reshaped_scale(self, X_shape): 
        new_shape = (self.scale_.shape[0],) + (1,) * (len(X_shape) - 1)
        return np.reshape(self.scale_, new_shape)
    
    def transform(self, X): 
        if self.copy: 
            X = X.copy()
        
        scale_ = self._reshaped_scale(X.shape)
        X /= scale_ 
        return X
    
    def inverse_transform(self, X): 
        if self.copy: 
            X = X.copy()
        
        scale_ = self._reshaped_scale(X.shape)
        X *= scale_
        return X 
    
    def fit_transform(self, X): 
        return self.fit(X).transform(X)

Let's plot some data and highlight several sliding windows it it. Есть и тренд и сезонность. окно в 18 точек, слайдит слева направо по окну и 

In [ ]:
df.head()

In [ ]:
#https://www.kaggle.com/jcolless/m5-rnn-wavenet-n-beats-approach#N-BEATS

In [ ]:
y = df['sales'].values
window_size = 28
y_windows = create_sliding_windows(y, window_size)

_, ax = plt.subplots(1, figsize=(10, 5))
ax.plot(np.arange(y.shape[0]), y)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for i, window_idx in enumerate([10, 15, 50, 120]): 
    p = patches.Rectangle((window_idx, np.min(y)), window_size, 600, linewidth=2, edgecolor=colors[i], linestyle='--', facecolor='none')
    ax.add_patch(p)

Now, we need to have some way to create train/validation partitions.
Ideally we would use something line sklearn.model_selection.TimeSeriesSplit, which allows to create several backtest and do something very similar to traditional cross validation. But we are going to use single backtest here to keep things simple.

Note we don't do shuffling on this function cause it will break "time awareness" in dataset and we will end up learning on future to predict the past.

In [ ]:
# 
def train_validation_split(X, y, validation_pct=0.1): 
    size = X.shape[0]
    validation_rows = int(size * validation_pct) 
    
    index = np.arange(size)
    validation = index[-validation_rows:]
    training = index[:-validation_rows] 
    
    return X[training], y[training], X[validation], y[validation]

#### Prepare simple baselines.

We will use 2 very simple baselines: 
- **latest naive baseline**: outputs last known value (e.g. If we have data up until Sunday and want to predict for next week - each day gets Sunday's value as prediction)
- **seasonal naive baseline**: outputs last known seasonal value (e.g. If we have data up until Sunday and want to predict for next week - future Monday get's last known Monday value as prediction, Tuesday - last known Tuesday, etc.)

In [ ]:
# latest 
def simple_baseline(x, y):
    """ Simple naive baseline predictions. 
    For each row in y predicts latest known value. 
    """
    latest = x[:, -1, :]
    y_pred = np.tile(latest, (1, y.shape[1]))
    return y_pred


def seasonal_baseline(x, y, period): 
    """ Seasonal naive baseline predictions. 
    """
    indexes = np.arange(y.shape[1]) - period
    return np.squeeze(x[:, indexes])

Prepare evaluation metrics.
MASE: mean absolute scaled error
sMAPE: scaled mean absolute percentage error
OWA: overall weighted average (M4 competition metric)
RMSE

In [ ]:
def mase(y_true, y_pred, y_naive_pred):
    baseline_mae = mean_absolute_error(y_true, y_naive_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return mae / baseline_mae


def smape(y_true, y_pred):
    error = np.abs(y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred)) / 2)
    error[~np.isfinite(error)] = 0.0
    return error.sum() * 100.0 / len(y_true)


def owa(smape, smape_naive, mase, mase_naive): 
    return (smape / smape_naive + mase / mase_naive) / 2.


def rmse(y_true, y_pred): 
    return np.sqrt(mean_squared_error(y_true, y_pred))


def evaluate(y_true, y_pred, y_pred_naive): 
    RMSE = rmse(y_true, y_pred)
    sMAPE = smape(y_true, y_pred)
    sMAPE_naive = smape(y_true, y_pred_naive)
    MASE = mase(y_true, y_pred, y_pred_naive)
    MASE_naive = mase(y_true, y_pred_naive, y_pred_naive)
    OWA = owa(sMAPE, sMAPE_naive, MASE, MASE_naive)
    
    print('\tRMSE   {}'.format(RMSE))
    print('\tsMAPE  {}'.format(sMAPE))
    print('\tMASE   {}'.format(MASE))
    print('\tOWA    {}'.format(OWA))

Sanity check: evaluating baselines

In [ ]:
data = df['sales'].values
fdw = 28*4
fw = 28
data_windows = create_sliding_windows(data, window_size=fdw + fw)
X = data_windows[:, :fdw, np.newaxis]
y = data_windows[:, fdw:]

X_train, y_train, X_val, y_val = train_validation_split(X, y, validation_pct=0.2)

y_pred_simple = simple_baseline(X_val, y_val)
y_pred_seasonal = seasonal_baseline(X_val, y_val, period=28)# один спайк это год, сезонность нужно знать наперед

print('Simple baseline')
evaluate(y_val, y_pred_simple, y_pred_simple)

print('Seasonal baseline')
evaluate(y_val, y_pred_seasonal, y_pred_seasonal)

plot_predictions(X_val[0], y_val[0], y_pred_simple[0])
plot_predictions(X_val[0], y_val[0], y_pred_seasonal[0])

Time to build models!
Let's start with very simple seq2seq-like network, which has RNN encoder, RNN decoder and linear output layer.

In [ ]:
def create_rnn_cells(rnn_units):
    cells = []
    for units in rnn_units:
        cells.append(keras.layers.LSTMCell(units))
    return cells

#Модель!
class LSTMEstimator(keras.Model): 
    def __init__(self, fw, rnn_units, output_activation='linear'): 
        super(LSTMEstimator, self).__init__()
        
        self.encoder_layer = keras.layers.RNN(create_rnn_cells(rnn_units))
        self.repeat_layer = keras.layers.RepeatVector(fw)#повторить на кол-во предикшинов с прошлого шага
        self.decoder_layer = keras.layers.RNN(create_rnn_cells(rnn_units), return_sequences=True)
        self.output_layer = keras.layers.Dense(1, activation=output_activation)
        
    def call(self, inputs): #модефайним как лояра друг за другом екзекьютятся
        outputs = self.encoder_layer(inputs)
        outputs = self.repeat_layer(outputs)
        outputs = self.decoder_layer(outputs)
        outputs = self.output_layer(outputs)
        return outputs

In [ ]:
model = LSTMEstimator(fw=28, rnn_units=[32])
model.compile(loss='mae', optimizer=keras.optimizers.Adam(0.01))#дефолтное значение
history = model.fit(
    X_train, 
    y_train,
    validation_data=(X_val, y_val), 
    epochs=200, 
    batch_size=8,
    shuffle=True, 
    verbose=False,
)
plot_loss(history.history)

In [ ]:
y_pred = model.predict(X_val)
i = 10
plot_predictions(X_val[i], y_val[i], y_pred[i])
evaluate(y_val, np.squeeze(y_pred), y_pred_seasonal)

In [ ]:
train_scaler = MeanScaler()
train_scaler.fit(X_train)

val_scaler = MeanScaler()
val_scaler.fit(X_val)

model = LSTMEstimator(fw=12, rnn_units=[32])
model.compile(loss='mae', optimizer=keras.optimizers.Adam(0.01))
history = model.fit(
    train_scaler.transform(X_train), 
    train_scaler.transform(y_train),
    validation_data=(val_scaler.transform(X_val), val_scaler.transform(y_val)), 
    epochs=200, 
    batch_size=8,
    shuffle=True, 
    verbose=False,
)
plot_loss(history.history)

In [ ]:
y_pred = model.predict(val_scaler.transform(X_val))
y_pred = val_scaler.inverse_transform(y_pred)
y_pred_seasonal = seasonal_baseline(X_val, y_val, period=12)
i = 10
plot_predictions(X_val[i], y_val[i], y_pred[i])
evaluate(y_val, np.squeeze(y_pred), y_pred_seasonal)

### DeepAR  по воркшопу https://www.youtube.com/watch?v=OP4wa3cFSG8&list=PLmFBTSyQ2rzUy1K929-6RRfFcfDZPUq6U&index=2

In [ ]:
class GaussianLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(GaussianLayer, self).__init__(**kwargs)
        self.W_mu = None
        self.b_mu = None
        self.W_sigma = None
        self.b_sigma = None

    def build(self, input_shape):
        super(GaussianLayer, self).build(input_shape)
        
        dim = input_shape[-1]
        
        self.W_mu = self.add_weight(
            name='W_mu', 
            shape=(dim, 1), 
            initializer='glorot_normal', 
            trainable=True,
        )
        self.b_mu = self.add_weight(
            name='b_mu', 
            shape=(1,), 
            initializer='zeros', 
            trainable=True,
        )
        
        self.W_sigma = self.add_weight(
            name='W_sigma',
            shape=(dim, 1),
            initializer='glorot_normal',
            trainable=True,
        )
        self.b_sigma = self.add_weight(
            name='b_sigma',
            shape=(1,),
            initializer='zeros', 
            trainable=True,
        )        
        
    def call(self, inputs):
        mu = K.dot(inputs, self.W_mu)
        mu = K.bias_add(mu, self.b_mu, data_format='channels_last')
        
        sigma = K.dot(inputs, self.W_sigma)
        sigma = K.bias_add(sigma, self.b_sigma, data_format='channels_last')
        sigma = K.softplus(sigma) + K.epsilon()
        
        return tf.squeeze(mu, axis=-1), tf.squeeze(sigma, axis=-1) 
    

def gaussian_loss(y_true, mu, sigma):
    loss = (
         tf.math.log(sigma) 
        + 0.5 * tf.math.log(2 * np.pi) 
        + 0.5 * tf.square(tf.math.divide(y_true - mu, sigma))
    )
    return tf.reduce_mean(loss)

    
def gaussian_sample(mu, sigma): 
    mu = tf.expand_dims(mu, axis=-1)
    sigma = tf.expand_dims(sigma, axis=-1)
    
    samples = tf.random.normal((300,), mean=mu, stddev=sigma)
    return tf.reduce_mean(samples, axis=-1)

    
class DeepAR(keras.Model): 
    def __init__(self, fw, rnn_units): 
        super(DeepAR, self).__init__()
        
        self.encoder_layer = keras.layers.RNN(create_rnn_cells(rnn_units))
        self.repeat_layer = keras.layers.RepeatVector(fw)
        self.decoder_layer = keras.layers.RNN(create_rnn_cells(rnn_units), return_sequences=True)
        self.gaussian_layer = GaussianLayer()

    def call(self, inputs): 
        outputs = self.encoder_layer(inputs)
        outputs = self.repeat_layer(outputs)
        outputs = self.decoder_layer(outputs)
        return self.gaussian_layer(outputs)

    def train_step(self, inputs):
        x, y = inputs
        
        with tf.GradientTape() as tape: 
            mu, sigma = self(x)
            loss_val = self.loss(y, mu, sigma)

        grads = tape.gradient(loss_val, self.trainable_weights)
        grads = [tf.clip_by_value(grad, -1., 1.) for grad in grads]
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        return {'loss': loss_val}

    def test_step(self, inputs):
        x, y = inputs
        mu, sigma = self(x, training=False)
        loss_val = self.loss(y, mu, sigma)
        return {'loss': loss_val}
    
    def predict_step(self, inputs): 
        x, = inputs
        mu, sigma = self(x, training=False)
        return gaussian_sample(mu, sigma)

In [ ]:
rnn_units = [32]

deep_ar = DeepAR(fw, rnn_units)

train_scaler = MeanScaler()
train_scaler.fit(X_train)

val_scaler = MeanScaler()
val_scaler.fit(X_val)

deep_ar.compile(loss=gaussian_loss, optimizer=keras.optimizers.Adam(0.01))

history = deep_ar.fit(
    train_scaler.transform(X_train), 
    train_scaler.transform(y_train),
    validation_data=(val_scaler.transform(X_val), val_scaler.transform(y_val)), 
    epochs=200, 
    batch_size=8,
    shuffle=True, 
    verbose=False,
)
plot_loss(history.history)

In [ ]:
y_pred = deep_ar.predict(val_scaler.transform(X_val))
y_pred = val_scaler.inverse_transform(y_pred)
i = 10
plot_predictions(X_val[i], y_val[i], y_pred[i])
evaluate(y_val, np.squeeze(y_pred), y_pred_seasonal)

Modeling Poisson distribution

In [ ]:
class PoissonLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(PoissonLayer, self).__init__(**kwargs)
        self.W_rate = None
        self.b_rate = None

    def build(self, input_shape):
        super(PoissonLayer, self).build(input_shape)
        
        dim = input_shape[0][-1]
        
        self.W_rate = self.add_weight(
            name='W_rate', 
            shape=(dim, 1), 
            initializer='glorot_uniform', 
            trainable=True,
        )
        self.b_rate = self.add_weight(
            name='b_rate', 
            shape=(1,), 
            initializer='zeros', 
            trainable=True,
        )

    def call(self, inputs):
        x, scale = inputs
        
        rate = K.dot(x, self.W_rate)
        rate = K.bias_add(rate, self.b_rate, data_format='channels_last')
        rate = K.softplus(rate) + K.epsilon()
        rate = rate * scale
        
        return tf.squeeze(rate, axis=-1)


class MeanScalerLayer(keras.layers.Layer): 
    
    def call(self, inputs): 
        scale = tf.reduce_mean(tf.abs(inputs), axis=1, keepdims=True) + 1.0
        outputs = inputs / scale
        return outputs, scale


def poisson_loss(y_true, rate): 
    loss = -1.0 * (
        y_true * tf.math.log(rate) 
        - tf.math.lgamma(y_true + 1.0) 
        - rate
    )
    return tf.reduce_mean(loss)


def poisson_sample(rate): 
    sample = tf.random.poisson((300,), lam=rate)
    sample = tf.reduce_mean(sample, axis=0)
    return tf.expand_dims(sample, axis=-1)


class DeepARPos(keras.Model): 
    def __init__(self, fw, rnn_units): 
        super(DeepARPos, self).__init__()
        
        self.mean_scaler_layer = MeanScalerLayer()
        self.encoder_layer = keras.layers.RNN(create_rnn_cells(rnn_units))
        self.repeat_layer = keras.layers.RepeatVector(fw)
        self.decoder_layer = keras.layers.RNN(create_rnn_cells(rnn_units), return_sequences=True)
        self.poisson_layer = PoissonLayer()

    def call(self, inputs): 
        outputs, scale = self.mean_scaler_layer(inputs)
        outputs = self.encoder_layer(outputs)
        outputs = self.repeat_layer(outputs)
        outputs = self.decoder_layer(outputs)
        return self.poisson_layer([outputs, scale])

    def train_step(self, inputs):
        x, y = inputs
        
        with tf.GradientTape() as tape: 
            rate = self(x)
            loss_val = self.loss(y, rate)
    
        grads = tape.gradient(loss_val, self.trainable_weights)
        grads = [tf.clip_by_value(grad, -1., 1.) for grad in grads]
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        return {'loss': loss_val}

    def test_step(self, inputs):
        x, y = inputs
        rate = self(x, training=False)
        loss_val = self.loss(y, rate)
        return {'loss': loss_val}
    
    def predict_step(self, inputs): 
        x, = inputs
        rate = self(x, training=False)
        return poisson_sample(rate)

In [ ]:
rnn_units = [32]

deep_ar = DeepARPos(fw, rnn_units)
deep_ar.compile(loss=poisson_loss, optimizer=keras.optimizers.Adam(0.01))

history = deep_ar.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val), 
    epochs=200, 
    batch_size=8,
    shuffle=True, 
    verbose=False,
)
plot_loss(history.history)

In [ ]:
y_pred = deep_ar.predict(X_val)

i = 10
plot_predictions(X_val[i], y_val[i], y_pred[i])
evaluate(y_val, np.squeeze(y_pred), y_pred_seasonal)